In [1]:
import time
import pandas as pd
import numpy as np
#install these
#!pip3 install lightgbm 
#!pip3 install catboost
#!pip3 install xgboost
#!pip3 install nltk
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

In [2]:
pd.set_option('max_colwidth', None)

df = pd.read_csv("reduced_123.csv", names=['category','rating', 'label', 'text'])
df['text'] = df['text'].str.replace('\n', ' ')
df['target'] = np.where(df['label']=='CG', 1, 0)
df['target'].value_counts()

0    35907
Name: target, dtype: int64

In [3]:
def punctuation_to_features(df, column):
    """Identify punctuation within a column and convert to a text representation.
    
    Args:
        df (object): Pandas dataframe.
        column (string): Name of column containing text. 
        
    Returns:
        df[column]: Original column with punctuation converted to text, 
                    i.e. "Wow! > "Wow exclamation"
    
    """
    
    df[column] = df[column].replace('!', ' exclamation ')
    df[column] = df[column].replace('?', ' question ')
    df[column] = df[column].replace('\'', ' quotation ')
    df[column] = df[column].replace('\"', ' quotation ')
    
    return df[column]

In [4]:
nltk.download('punkt');

def tokenize(column):
    """Tokenizes a Pandas dataframe column and returns a list of tokens.
    
    Args:
        column: Pandas dataframe column (i.e. df['text']).
    
    Returns:
        tokens (list): Tokenized list, i.e. [Donald, Trump, tweets]
    
    """
    
    tokens = nltk.word_tokenize(column)
    return [w for w in tokens if w.isalpha()] 

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
df['tokenized'] = df.apply(lambda x: tokenize(x['text']), axis=1)
df.head()

,,category,rating,label,text,target,tokenized
User_id,Product_id,Rating,Date,Review,Label,0,[Label]
936,0,5,7/7/2008,Perfection. Â Snack has become my favorite late lunch/early dinner spot. Â Make sure to try the butter beans!!!,-1,0,[]
940,0,4,9/16/2014,"For lunch, my friend and I had: -Lamb sandwich, juicy and flavorful. -Roasted beet salad, pretty big portion. Good thing my friend shared it with me; otherwise I think I would so bored eating only beets, and a few leaves of greens. -Classic Greek Avolemono, basically a very tangy chicken broth with orzo. My friend really likes it at this place. The place is small. There are only 6 tables, but they do delivery. Good service if you decide to dine in.",1,0,[]
963,0,5,3/2/2013,"This tiny cafe on Thompson has been a favorite of mine for years, and I'll tell you why. Everything is fresh and the attention to detail makes it a keeper. The lamb sammie on ciabatta is melt in your mouth chunks of lamb, with roasted onions and pretty much to die for. It's big enough for two gals to share, for sure. My favorite soup of all time, they rock here -- the Avgolemono is super lemony and perfect, with al dente orzo. Served with toasted slivers of olive oil coated fresh bread. And in every take away bag then toss little twists of waxed paper filled with a few Jordan almonds. Pretty adorable, since I'm a sucker for these little touches.",1,0,[]
965,0,4,2/14/2013,"Ohhh Snack. I love you, Snack. I order from here often since it's close to my work and their delivery is always precise and on-time. Their avgolemono and vegetarian souvlaki are fantastic and I hope that one day, maybe, I'll be able to replicate their giant beans in that delicious dill tomato sauce. I haven't yet eaten in the store itself, but I'm sure it'll be just as enjoyable as digging into my brown paper sack for the best (and still reasonably priced) lunch I can find in SoHo.",1,0,[]


In [8]:
nltk.download('stopwords');
def remove_stopwords(tokenized_column):
    """Return a list of tokens with English stopwords removed. 
    
    Args:
        column: Pandas dataframe column of tokenized data from tokenize()
    
    Returns:
        tokens (list): Tokenized list with stopwords removed.
    
    """
    stops = set(stopwords.words("english"))
    return [word for word in tokenized_column if not word in stops]

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
df['stopwords_removed'] = df.apply(lambda x: remove_stopwords(x['tokenized']), axis=1)
df.head()

,,category,rating,label,text,target,tokenized,stopwords_removed
User_id,Product_id,Rating,Date,Review,Label,0,[Label],[Label]
936,0,5,7/7/2008,Perfection. Â Snack has become my favorite late lunch/early dinner spot. Â Make sure to try the butter beans!!!,-1,0,[],[]
940,0,4,9/16/2014,"For lunch, my friend and I had: -Lamb sandwich, juicy and flavorful. -Roasted beet salad, pretty big portion. Good thing my friend shared it with me; otherwise I think I would so bored eating only beets, and a few leaves of greens. -Classic Greek Avolemono, basically a very tangy chicken broth with orzo. My friend really likes it at this place. The place is small. There are only 6 tables, but they do delivery. Good service if you decide to dine in.",1,0,[],[]
963,0,5,3/2/2013,"This tiny cafe on Thompson has been a favorite of mine for years, and I'll tell you why. Everything is fresh and the attention to detail makes it a keeper. The lamb sammie on ciabatta is melt in your mouth chunks of lamb, with roasted onions and pretty much to die for. It's big enough for two gals to share, for sure. My favorite soup of all time, they rock here -- the Avgolemono is super lemony and perfect, with al dente orzo. Served with toasted slivers of olive oil coated fresh bread. And in every take away bag then toss little twists of waxed paper filled with a few Jordan almonds. Pretty adorable, since I'm a sucker for these little touches.",1,0,[],[]
965,0,4,2/14/2013,"Ohhh Snack. I love you, Snack. I order from here often since it's close to my work and their delivery is always precise and on-time. Their avgolemono and vegetarian souvlaki are fantastic and I hope that one day, maybe, I'll be able to replicate their giant beans in that delicious dill tomato sauce. I haven't yet eaten in the store itself, but I'm sure it'll be just as enjoyable as digging into my brown paper sack for the best (and still reasonably priced) lunch I can find in SoHo.",1,0,[],[]
